# KBO 1982~2020 투수 기록 크롤링(statiz.co.kr)

In [1]:
# 필요한 라이브러리 import
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd

# setup Driver | Chrome: 크롬 드라이버를 사용하는 driver 생성
driver = webdriver.Chrome('./chromedriver')
driver.implicitly_wait(10)

result = []
for i in range(1982, 2021):  # 시작년도, 종료년도 + 1
    count = 0
    for j in range(20):  # 19회 반복(*30 = 최대 570 Row 가져옴)   
        # url 접근
        driver.get('http://www.statiz.co.kr/stat.php?mid=stat&re=1&ys=' + str(i) + '&ye=' + str(i) + '&se=0&te=&tm=&ty=0&qu=auto&po=0&as=&ae=&hi=&un=&pl=&da=1&o1=WAR&o2=OutCount&de=1&lr=0&tr=&cv=&ml=1&sn=30&pa=' + str(j * 30) + '&si=&cn=')
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        tr = soup.select('table#mytable.table.table-striped.table-responsive.table-condensed.no-space.table-bordered > tbody > tr')
        if(len(tr) == 0):  # 선수 데이터 row 수가 0인 경우 조기 종료
            break
        # Row 수 계산
        for r in tr:
            row = []
            td = r.find_all('td')
            for d in td:
                row.append(d.text.strip())
            if(len(row) > 0):
                count += 1  # Row 수 세기
                result.append(row)
    # Row 수 출력
    print(i, '년: ', count, sep='')

data = pd.DataFrame(result)

1982년: 43
1983년: 55
1984년: 59
1985년: 69
1986년: 89
1987년: 88
1988년: 92
1989년: 119
1990년: 117
1991년: 136
1992년: 133
1993년: 124
1994년: 141
1995년: 140
1996년: 144
1997년: 144
1998년: 135
1999년: 151
2000년: 154
2001년: 175
2002년: 172
2003년: 171
2004년: 178
2005년: 175
2006년: 172
2007년: 172
2008년: 187
2009년: 192
2010년: 191
2011년: 186
2012년: 188
2013년: 194
2014년: 211
2015년: 244
2016년: 250
2017년: 239
2018년: 260
2019년: 257
2020년: 189


In [2]:
# 중복된 column(WAR) 제거
data = data.drop(data.columns[3], axis='columns')

In [3]:
# 한글 column
data.columns = ['순서', '이름', '팀', '출장', '완투', '완봉', '선발', '승리', '패전', '세이브', '홀드', '이닝', '실점', '자책', '상대 타자', '안타', '2루타', '3루타', '피홈런', '볼넷', '고의사구', '사구', '탈삼진', '보크', '폭투', 'ERA', 'FIP', 'WHIP', 'ERA+', 'FIP+', 'WAR', 'WPA']
data.to_csv('KBO_1982_2020_Pitcher_KR.csv', encoding='cp949', index=False)
# 영어 column
data.columns = ['No', 'Name', 'Team', 'G', 'CG', 'ShO', 'GS', 'W', 'L', 'SV', 'HLD(HD)', 'IP', 'R', 'ER', 'TBF', 'H', '2B', '3B', 'HR', 'BB', 'IBB', 'HBP', 'SO', 'BK', 'WP', 'ERA', 'FIP', 'WHIP', 'ERA+', 'FIP+', 'WAR', 'WPA']
data.to_csv('KBO_1982_2020_Pitcher_EN.csv', encoding='cp949', index=False)

## 특정 해 데이터가 유실된 경우

In [82]:
# 유실된 데이터 년도
year = 2007

restore_result = []
restore_count = 0
for j in range(20):  # 19회 반복(*30 = 최대 570 Row 가져옴)   
    # url 접근
    driver.get('http://www.statiz.co.kr/stat.php?mid=stat&re=1&ys=' + str(year) + '&ye=' + str(year) + '&se=0&te=&tm=&ty=0&qu=auto&po=0&as=&ae=&hi=&un=&pl=&da=1&o1=WAR&o2=OutCount&de=1&lr=0&tr=&cv=&ml=1&sn=30&pa=' + str(j * 30) + '&si=&cn=')
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    tr = soup.select('table#mytable.table.table-striped.table-responsive.table-condensed.no-space.table-bordered > tbody > tr')
    if(len(tr) == 0):  # 선수 데이터 row 수가 0인 경우 조기 종료
        break
    # Row 수 계산
    for r in tr:
        row = []
        td = r.find_all('td')
        for d in td:
            row.append(d.text.strip())
        if(len(row) > 0):
            restore_count += 1  # Row 수 세기
            restore_result.append(row)

# Row 수 출력
print(year, '년: ', restore_count, sep='')

restore_data = pd.DataFrame(restore_result)

2007년: 277


In [83]:
# 중복된 column(WAR) 제거
restore_data = restore_data.drop(restore_data.columns[3], axis='columns')

In [85]:
# 한글 column
restore_data.columns = ['순서', '이름', '팀', '출장', '완투', '완봉', '선발', '승리', '패전', '세이브', '홀드', '이닝', '실점', '자책', '상대 타자', '안타', '2루타', '3루타', '피홈런', '볼넷', '고의사구', '사구', '탈삼진', '보크', '폭투', 'ERA', 'FIP', 'WHIP', 'ERA+', 'FIP+', 'WAR', 'WPA']
restore_data.to_csv('KBO_' + str(year) + '_Pitcher_KR.csv', encoding='cp949', index=False)
# 영어 column
restore_data.columns = ['No', 'Name', 'Team', 'G', 'CG', 'ShO', 'GS', 'W', 'L', 'SV', 'HLD(HD)', 'IP', 'R', 'ER', 'TBF', 'H', '2B', '3B', 'HR', 'BB', 'IBB', 'HBP', 'SO', 'BK', 'WP', 'ERA', 'FIP', 'WHIP', 'ERA+', 'FIP+', 'WAR', 'WPA']
restore_data.to_csv('KBO_' + str(year) + '_Pitcher_EN.csv', encoding='cp949', index=False)